In [1]:
import pandas as pd

#### Leitura da base em JSON

In [2]:
# Ler arquivo Json - arquivo usando com orient=records
data = pd.read_json('definedcommentsdf.json')

# Cria um dataframe so para os valores das colunas toxico e comentario
comments_toxico = data.drop(['link','datahora'], axis=1)
comments_toxico.head()

,idcomentario,toxico,comentario,tipo
0,5e52b6f032766bbb6d01c5cc,False,Passando o pacote anti crime na camara e sena...,
1,5e52b83c32766bbb6d01d2c0,False,nossa ai fica dificil...antes sobia 20 reais ...,
2,5e52b83c32766bbb6d01d1e2,False,Até quando ele vai comparar tudo a um casamen...,
3,5e52b84f32766bbb6d01db26,False,99 nunca mais! Usei o 99 para pagar com cartã...,
4,5e52b84f32766bbb6d01d883,False,Já foi confirmado pela UFBA que é óleo venezu...,


In [3]:
comments_toxico['toxico'].value_counts()

False    1565
True      787
Name: toxico, dtype: int64

## Aqui começa o tratamento da base

In [4]:
# Modifica valores false/true para 0,1
comments_toxico.loc[(comments_toxico.toxico == True),'toxico'] = 1
comments_toxico.loc[(comments_toxico.toxico == False),'toxico'] = 0

In [5]:
# DataFrame com toxicos e não toxicos
toxic_data = comments_toxico[comments_toxico.toxico == 1]
nontoxic_data = comments_toxico[comments_toxico.toxico == 0]

In [6]:
# Conta quantidade de cada classificacao
count_toxic = toxic_data.shape[0]
count_nontoxic = nontoxic_data.shape[0]

# Checa a quantidade de cada dataset e deixa 50%/50% de cada classificacao 
if count_nontoxic > count_toxic:
    nontoxic_data = nontoxic_data.sample(count_toxic, replace=True)
    
if count_toxic > count_nontoxic:
    toxic_data = toxic_data.sample(count_nontoxic, replace=True)
    
# Ignora Warnings de indexação
pd.options.mode.chained_assignment = None

# Cria contagem de idcomentarios iguais
nontoxic_data['amount_idcomentario']= nontoxic_data.idcomentario.map(nontoxic_data.idcomentario.value_counts())
toxic_data['amount_idcomentario']= toxic_data.idcomentario.map(toxic_data.idcomentario.value_counts())

# Criar listas de apoio
data_bigtoxic = [] 
data_bignontoxic = []
dataigual = []
for row in nontoxic_data.itertuples():
    for row1 in toxic_data.itertuples():
        if row1.idcomentario == row.idcomentario and row1.amount_idcomentario > row.amount_idcomentario:
            data_bigtoxic.append(row1)
        elif row1.idcomentario == row.idcomentario and row1.amount_idcomentario < row.amount_idcomentario:
            data_bignontoxic.append(row)
        elif row1.idcomentario == row.idcomentario and row1.amount_idcomentario == row.amount_idcomentario:
            dataigual.append(row1.idcomentario)

In [7]:
# Recria dataframe com samples de toxic e non toxic, resetando index 
mergedata = [nontoxic_data,toxic_data]
new_dataframe = pd.concat(mergedata)
new_dataframe.reset_index(drop=True, inplace=True)

#### Caso queira remover comentário que tem notação tóxica e não tóxica 

In [8]:
# Remove comentarios que tem mesmo valor de toxico e nao toxico
# data_framefinal = new_dataframe[~new_dataframe.idcomentario.isin(dataigual)]

In [9]:
# remove itens duplicados
# comentarios_df = data_framefinal.drop(['idcomentario','tipo','amount_idcomentario'], axis=1)
# comentarios_df = comentarios_df.drop_duplicates(subset=['toxico', 'comentario'], keep='first')
# comentarios_df

In [10]:
comentarios_df = new_dataframe.drop(['idcomentario','tipo','amount_idcomentario'], axis=1)
comentarios_df['toxico'].value_counts()

1    787
0    787
Name: toxico, dtype: int64

In [11]:
# Transforma tóxico em int
comentarios_df["toxico"] = pd.to_numeric(comentarios_df["toxico"])

In [12]:
# Gera um Json com o dataframe com tratamento 
with open('dataset_for_modeling.json', 'w', encoding='utf-8') as file:
    comentarios_df.to_json(file, force_ascii=False, orient='records')